# Bumblebee
Easy

**Scenario**  
An external contractor has accessed the internal forum here at Forela via the Guest WiFi and they appear to have stolen credentials for the administrative user! We have attached some logs from the forum and a full database dump in sqlite3 format to help you in your investigation.

## Evidence
1. Log file
1. PHPBB SQLITE3 database

## Solutions
- Task 1: What was the username of the external contractor?
    - Shown in db table `phpbb_users`, which lists two users as contractors
    - Either  `apoole` or `apoole1`
    - Answer: `apoole1`
- Task 2: What IP address did the contractor use to create their account?
    - Shown in th db same table as above, in column `user_ip`
    - Answer: `10.10.0.78`
- Task 3: What is the post_id of the malicious post that the contractor made?
    - Shown in db table `phpbb_posts`, which lists three posts
    - The malicious post is the one, in which column `post_text` contains a large HTML formatted text
    - Answer: `9`
- Task 4: What is the full URI that the credential stealer sends its data to?
    - Copying the HTML from previous answer into a HTML beautyfier online, we get a more readable HTML text
    - This shows a form, which submits data to `http://10.10.0.78/update.php`
    - Answer: `http://10.10.0.78/update.php`
- Task 5: When did the contractor log into the forum as the administrator? (UTC)
    - The answer resides in the database table `phpbb_log`, and the login events of interest are `log_id` 52 and 61
    - The timestamp for these events are found in column `log_time` and must be converted to UTC timestamps, see code below
    - The correct `log_id` is 61, with `log_time` = `1682506392`
    - Answer: `26/04/2023 10:53:12`
- Task 6: In the forum there are plaintext credentials for the LDAP connection, what is the password?
    - This is stored in the database, table `phpbb_config`, under the `config_name` `ldap_password`
    - Answer: `Passw0rd1`
- Task 7: What is the user agent of the Administrator user?
    - Parsing the access logs gave three User agents, of which this matched
    - Answer: `Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/112.0.0.0 Safari/537.36`
- Task 8: What time did the contractor add themselves to the Administrator group? (UTC)
    - The answer resides in the database table `phpbb_log`, and the login events of interest are `log_id` 62, with `log_time` = `1682506431`
    - Answer: `26/04/2023 10:53:51`
- Task 9: What time did the contractor download the database backup? (UTC)
    - Looking at the access log file, we see the initialization of two downloads
    - The "end" log entry is the one without a URL, and a large size field
    - Answer: `26/04/2023 11:01:38`
- Task 10: What was the size in bytes of the database backup as stated by access.log?
    - Using the same entry in the access log we get the `size`
    - Answer: `34707`

In [ ]:
#Code for converting UNIX timestamps to correct format for answers
from datetime import datetime, timezone
print(datetime.strftime(datetime.fromtimestamp(1682506431, tz=timezone.utc), "%d/%m/%Y %H:%M:%S"))

In [ ]:
import csv
from datetime import datetime, timezone
import pandas as pd
data = []
with open(r"C:\Users\magnu\Downloads\access.log") as log_fh:
    lines = csv.reader(log_fh, delimiter=" ", quotechar='"', )
    for line in lines:
        for index,val in enumerate(line):
            if val == "-":
                line[index] = None
        ts_zone = line.pop(4)
        ts_time = line.pop(3)
        ts_datetime = datetime.strptime(f"{ts_time[1:]}{ts_zone[:-1]}", "%d/%b/%Y:%H:%M:%S%z")
        line.insert(3, ts_datetime.astimezone(timezone.utc))
        request = line.pop(4).split(" ")
        line.insert(4, request[0])
        line.insert(5, request[1])
        line.insert(6, request[2])
        line.pop(1)
        line.pop(1)
        data.append(line)
df = pd.DataFrame(data, columns=["ip", "ts", "HTTP_method", "request", "HTTP_version", "response_code", "size", "URL", "user_agent"])
show_case = ["ip", "ts", "HTTP_method", "response_code", "size", "URL"]
for data in df.itertuples():
    print(f"{datetime.strftime(data.ts, "%d/%m/%Y %H:%M:%S")}  {data.HTTP_method:>10} {data.response_code:>6} {data.size:>6}   {data.URL}")